In [19]:
import mediapipe as mp
import numpy as np
import cv2
import time

In [20]:
import torch 
import torch.nn as nn
import torch_geometric
from torch_geometric.data import Data
import torch.optim as optimizer
device = torch.device('mps')


In [21]:
if torch.cuda.is_available():
    print("CUDA is available!")
    # CUDA 장치 이름 및 세부 정보 확인
    device_name = torch.cuda.get_device_name(0)
    print(f"Device: {device_name}")
else:
    print("CUDA is not available.")

CUDA is available!
Device: NVIDIA GeForce RTX 3060


In [22]:
mp_pose = mp.solutions.pose
def create_invalid_pose(keypoints, width, height, pose_type, frame_idx, total_frames):
    """
    특정 운동 유형에 따른 잘못된 자세를 생성하며, 프레임 길이에 따라 변형 정도를 조정합니다.

    Parameters:
        keypoints (list): Mediapipe 랜드마크 리스트 (정규화된 값).
        width (int): 영상의 너비 (픽셀 기준).
        height (int): 영상의 높이 (픽셀 기준).
        pose_type (str): 운동 유형 ("lunge", "pushup", "squat").
        frame_idx (int): 현재 프레임 인덱스.
        total_frames (int): 해당 영상의 총 프레임 수.
    
    Returns:
        invalid_keypoints_px (list): 변형된 자세의 픽셀 좌표 리스트.
    """
    invalid_keypoints = []

    # 변형 강도를 프레임 기반으로 조정
    progress = frame_idx / total_frames  # 진행률 (0에서 1 사이)
    intensity = abs(np.sin(progress * np.pi))  # 0 -> 1 -> 0 패턴 생성

    for idx, landmark in enumerate(keypoints):
        x, y, z = landmark.x, landmark.y, landmark.z  # 정규화된 좌표

        if pose_type == "lunge":
            if idx == mp_pose.PoseLandmark.LEFT_KNEE.value:
                # 왼쪽 무릎: 왼쪽으로 더 이동
                x -= intensity * np.random.uniform(0.02, 0.05)
                y += intensity * np.random.uniform(0.02, 0.05)
            elif idx == mp_pose.PoseLandmark.RIGHT_KNEE.value:
            # 오른쪽 무릎: 오른쪽으로 더 이동
                x += intensity * np.random.uniform(0.02, 0.05)
                y += intensity * np.random.uniform(0.02, 0.05)
            elif idx == mp_pose.PoseLandmark.LEFT_SHOULDER.value:
                x -= intensity * np.random.uniform(0.03, 0.05)  # 왼쪽 어깨
                y += intensity * np.random.uniform(-0.03, 0.03)
            elif idx == mp_pose.PoseLandmark.RIGHT_SHOULDER.value:
                x += intensity * np.random.uniform(0.03, 0.05)  # 오른쪽 어깨
                y += intensity * np.random.uniform(-0.03, 0.03)

        elif pose_type == "pushup":
            if idx == mp_pose.PoseLandmark.LEFT_ELBOW.value:
                x -= intensity * np.random.uniform(0.03, 0.05)  # 왼쪽 팔꿈치
                y += intensity * np.random.uniform(0.02, 0.05)
            elif idx == mp_pose.PoseLandmark.RIGHT_ELBOW.value:
                x += intensity * np.random.uniform(0.03, 0.05)  # 오른쪽 팔꿈치
                y += intensity * np.random.uniform(0.02, 0.05)
            elif idx == mp_pose.PoseLandmark.LEFT_HIP.value:
                x -= intensity * np.random.uniform(0.02, 0.04)  # 왼쪽 엉덩이
                y += intensity * np.random.uniform(-0.03, 0.03)
            elif idx == mp_pose.PoseLandmark.RIGHT_HIP.value:
                x += intensity * np.random.uniform(0.02, 0.04)  # 오른쪽 엉덩이
                y += intensity * np.random.uniform(-0.03, 0.03)

        elif pose_type == "squat":
            if idx == mp_pose.PoseLandmark.LEFT_KNEE.value:
                x -= intensity * np.random.uniform(0.03, 0.06)  # 왼쪽 무릎
                y += intensity * np.random.uniform(0.03, 0.03)
            elif idx == mp_pose.PoseLandmark.RIGHT_KNEE.value:
                x += intensity * np.random.uniform(0.03, 0.06)  # 오른쪽 무릎
                y += intensity * np.random.uniform(0.03, 0.03)
            elif idx == mp_pose.PoseLandmark.LEFT_HIP.value:
                x -= intensity * np.random.uniform(0.03, 0.05)  # 왼쪽 엉덩이
                y += intensity * np.random.uniform(-0.02, -0.05)
            elif idx == mp_pose.PoseLandmark.RIGHT_HIP.value:
                x += intensity * np.random.uniform(0.03, 0.05)  # 오른쪽 엉덩이
                y += intensity * np.random.uniform(-0.02, -0.05)
            elif idx == mp_pose.PoseLandmark.LEFT_SHOULDER.value:
                x -= intensity * np.random.uniform(0.03, 0.05)  # 왼쪽 어깨
                y += intensity * np.random.uniform(-0.02, -0.02)
            elif idx == mp_pose.PoseLandmark.RIGHT_SHOULDER.value:
                x += intensity * np.random.uniform(0.03, 0.05)  # 오른쪽 어깨
                y += intensity * np.random.uniform(-0.02, -0.02)


        invalid_keypoints.append((x, y, z))
    
    # 정규화된 좌표 -> 픽셀 좌표 변환
    invalid_keypoints_px = [
        (int(lm[0] * width), int(lm[1] * height)) for lm in invalid_keypoints
    ]

    return invalid_keypoints_px


## 검은 화면에 그래프만 남기기 - 영상 전체

In [18]:
# Mediapipe Pose 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.5)


# verbose 설정
verbose = False  # 상세 로그 출력 여부

# 비디오 파일 범위 처리 (운동_001.mp4부터 운동_100.mp4까지)
for i in range(1, 101):
    # 비디오 파일 경로 생성
    video_file = f'{data}/{data}_{i:03d}.{extension}'
    cap = cv2.VideoCapture(video_file)
    
    if not cap.isOpened():
        if verbose:
            print(f"Cannot open video {video_file}")
        continue
    
    # 영상 정보 가져오기
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 출력 파일 경로
    out_correct= f'data/{data}_processed/{data}_graph_{i:03d}.mp4'
    out_wrong = f'data/{data}_wrong/{data}_wrong_{i:03d}.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out= cv2.VideoWriter(out_correct, fourcc, fps, (w, h))
    out_incorrect = cv2.VideoWriter(out_wrong, fourcc, fps, (w, h))
    prev_time = 0
    frame_idx = int(0)
    # Mediapipe Pose 실행
    while cap.isOpened():
        success, image = cap.read()
        
        if not success or image is None:
            if verbose:
                print(f"End of video {video_file}")
            break
        
        curr_time = time.time()
        
        # BGR 이미지를 RGB로 변환 후 처리
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        # 검은 화면 생성
        black_frame_correct = np.zeros((h, w, 3), dtype=np.uint8)
        black_frame_incorrect = np.zeros((h, w, 3), dtype=np.uint8)
        # 랜드마크 그리기
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                black_frame_correct, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))
        if results.pose_landmarks:
            keypoints = results.pose_landmarks.landmark

            #올바른 자세 그래프
            mp_drawing.draw_landmarks(
            black_frame_correct, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
            )

    # 잘못된 자세 그래프
        invalid_keypoints_px = create_invalid_pose(keypoints, w, h,data,int(frame_idx), int(fps))
        for start, end in mp_pose.POSE_CONNECTIONS:
            start_px, end_px = invalid_keypoints_px[start], invalid_keypoints_px[end]
            cv2.line(black_frame_incorrect, start_px, end_px, (0, 255, 0), 2)

        for point in invalid_keypoints_px:
            cv2.circle(black_frame_incorrect, point, 3, (0, 255, 0), -1)
        frame_idx +=1
        #black_frame_correct = cv2.resize(black_frame_correct,(480,640))
        #black_frame_incorrect = cv2.resize(black_frame_incorrect,(480,640))
        # 출력 영상 저장 (화면 출력은 없음)
        out.write(black_frame_correct)
        out_incorrect.write(black_frame_incorrect)
    cap.release()
    out.release()
    out_incorrect.release()
cv2.destroyAllWindows()


In [ ]:
# 비디오 파일 범위 처리
for i in range(1, 101):
    video_file = f'{data}/{data}_{i:03d}.{extension}'
    cap = cv2.VideoCapture(video_file)
    
    if not cap.isOpened():
        if verbose:
            print(f"Cannot open video {video_file}")
        continue
    
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 출력 파일 경로
    out_correct_file = f'data/{data}_processed/{data}_correct_{i:03d}.mp4'
    out_incorrect_file = f'data/{data}_processed/{data}_incorrect_{i:03d}.mp4'
    
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out_correct = cv2.VideoWriter(out_correct_file, fourcc, fps, (w, h))
    out_incorrect = cv2.VideoWriter(out_incorrect_file, fourcc, fps, (w, h))
    
    while cap.isOpened():
        success, image = cap.read()
        if not success or image is None:
            if verbose:
                print(f"End of video {video_file}")
            break
        
        # Mediapipe Pose 처리
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        # 검은 화면 생성
        black_frame_correct = np.zeros((h, w, 3), dtype=np.uint8)
        black_frame_incorrect = np.zeros((h, w, 3), dtype=np.uint8)
        
        if results.pose_landmarks:
            # 랜드마크 가져오기
            landmarks = results.pose_landmarks.landmark
            
            # 올바른 자세 랜드마크 그리기
            mp_drawing.draw_landmarks(
                black_frame_correct, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))
            
            # 잘못된 자세 생성 및 그리기
            invalid_landmarks_px = create_invalid_pose(landmarks, w, h)
            for start, end in mp_pose.POSE_CONNECTIONS:
                start_px, end_px = invalid_landmarks_px[start], invalid_landmarks_px[end]
                cv2.line(black_frame_incorrect, start_px, end_px, (0, 255, 0), 2)
            
            for point in invalid_landmarks_px:
                cv2.circle(black_frame_incorrect, point, 3, (0,255, 0), -1)
        black_frame_correct 
        # 각각의 영상 저장
        out_correct.write(black_frame_correct)
        out_incorrect.write(black_frame_incorrect)
    
    cap.release()
    out_correct.release()
    out_incorrect.release()

cv2.destroyAllWindows()


## 검은 화면에 그래프만 남기기 - 영상 초기 15프레임만

In [13]:
# Mediapipe Pose 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# verbose 설정
verbose = False  # 상세 로그 출력 여부
data = "not"
extension = "mp4"  # 비디오 파일 확장자

# Pose 모델 초기화 (루프 바깥에서)
pose = mp_pose.Pose(static_image_mode=False, 
                    model_complexity=1, 
                    enable_segmentation=False, 
                    min_detection_confidence=0.5)

# 비디오 파일 범위 처리 (운동_001.mp4부터 운동_100.mp4까지)
for i in range(1, 101):
    # 비디오 파일 경로 생성
    video_file = f'{data}/{data}_{i:03d}.{extension}'
    cap = cv2.VideoCapture(video_file)
    
    if not cap.isOpened():
        if verbose:
            print(f"Cannot open video {video_file}")
        continue
    
    # 영상 정보 가져오기
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 출력 파일 경로
    out_file = f'data/{data}_start/{data}_graph_start_{i:03d}.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(out_file, fourcc, fps, (w, h))
    
    prev_time = 0
    frame_count = 0  # 프레임 카운트 변수
    
    # Mediapipe Pose 실행
    while cap.isOpened() and frame_count < 15:  # 15프레임까지만 처리
        success, image = cap.read()
        if not success:
            if verbose:
                print(f"End of video {video_file}")
            break
        
        curr_time = time.time()
        
        # 검은 화면 생성
        black_frame = np.zeros((h, w, 3), dtype=np.uint8)
        
        # BGR 이미지를 RGB로 변환 후 처리
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        # 랜드마크 그리기
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                black_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))
    
        # 출력 영상 저장 (화면 출력은 없음)
        out.write(black_frame)
        
        frame_count += 1  # 처리한 프레임 수 증가
    
    cap.release()
    out.release()

# Pose 모델 종료
pose.close()

cv2.destroyAllWindows()


I0000 00:00:1737334578.264220 55949771 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
OpenCV: Couldn't read video stream from file "not/not_001.mp4"
OpenCV: Couldn't read video stream from file "not/not_002.mp4"
OpenCV: Couldn't read video stream from file "not/not_003.mp4"
OpenCV: Couldn't read video stream from file "not/not_004.mp4"
OpenCV: Couldn't read video stream from file "not/not_005.mp4"
OpenCV: Couldn't read video stream from file "not/not_006.mp4"
OpenCV: Couldn't read video stream from file "not/not_007.mp4"
OpenCV: Couldn't read video stream from file "not/not_008.mp4"
OpenCV: Couldn't read video stream from file "not/not_009.mp4"
OpenCV: Couldn't read video stream from file "not/not_010.mp4"
OpenCV: Couldn't read video stream from file "not/not_011.mp4"
OpenCV: Couldn't read video stream from file "not/not_012.mp4"
OpenCV: Couldn't read video stream from file "not/not_013.mp4"
OpenCV: Couldn't read video stream from file "not/not_014.mp4"
OpenC

## 웹캠 영상 그래프로 변환

In [ ]:
# Mediapipe Pose 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, 
                    model_complexity=1, 
                    enable_segmentation=False, 
                    min_detection_confidence=0.5, 
                    min_tracking_confidence=0.5)

# 비디오 입력 설정
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Pose 추론
    results = pose.process(rgb_frame)
    
    # 검은 화면 생성
    black_frame = np.zeros((480, 640, 3), dtype=np.uint8)  # 480x640 해상도, 검은 배경
    
    # Pose 결과를 검은 화면에 그리기
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            black_frame, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),  # 랜드마크 스타일
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)  # 연결선 스타일
        )
    
    # 결과 화면 출력
    cv2.imshow('Pose Graph on Black Background', black_frame)
    cv2.imshow('video',frame)
    if cv2.waitKey(1) == ord('q'):  # ESC 키로 종료
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

## 영상에서 노드와 엣지 추출(그래프 데이터로)

In [21]:
data = 'push'

In [22]:
import json
import os
# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 데이터 폴더 설정
output_folder = f"{data}_graph_folder"  # 출력 폴더 (추출된 노드와 엣지를 저장)
# 출력 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 비디오 파일 경로 설정 (data_001.mp4부터 data_101.mp4까지 처리)
for i in range(1, 101):  # 1번부터 100번까지 처리
    video_file = f"{data}/{data}_{i:03d}.mov"  # 비디오 파일 경로 생성
    cap = cv2.VideoCapture(video_file)

    if not cap.isOpened():
        print(f"Cannot open video {video_file}")
        continue

    # JSON 파일 생성 (노드 및 엣지 저장)
    json_filename = os.path.join(output_folder, f"{data}_{i:03d}_landmarks_edges.json")
    all_frames_data = []  # 모든 프레임 데이터를 저장할 리스트

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # 이미지를 RGB로 변환
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(rgb_frame)

        frame_data = {"frame": frame_count, "nodes": [], "edges": []}  # 각 프레임 데이터 초기화

        # Pose landmarks 추출
        if result.pose_landmarks:
            # 각 부위 (노드)의 좌표 저장
            for i, landmark in enumerate(result.pose_landmarks.landmark):
                node_data = {"node_id": i, "x": landmark.x, "y": landmark.y, "z": landmark.z}
                frame_data["nodes"].append(node_data)

            # 엣지 추출: 인체 부위 간 연결
            edges = [
                (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW),
                (mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW),
                (mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST),
                (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
                # 추가적인 연결부위를 추가 가능
            ]

            # 엣지 정보 저장
            for edge in edges:
                start_node = result.pose_landmarks.landmark[edge[0]]
                end_node = result.pose_landmarks.landmark[edge[1]]
                edge_data = {"start_node": edge[0], "end_node": edge[1], 
                             "start_x": start_node.x, "start_y": start_node.y, "start_z": start_node.z,
                             "end_x": end_node.x, "end_y": end_node.y, "end_z": end_node.z}
                frame_data["edges"].append(edge_data)

        # 각 프레임 데이터를 리스트에 추가
        all_frames_data.append(frame_data)

    # JSON 파일에 전체 프레임 데이터 저장
    with open(json_filename, 'w') as json_file:
        json.dump(all_frames_data, json_file, indent=4)

    cap.release()

cv2.destroyAllWindows()


In [12]:
import cv2
print(cv2.getBuildInformation())


General configuration for OpenCV 4.10.0 =====================================
  Version control:               4.10.0

  Extra modules:
    Location (extra):            /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv_contrib/modules
    Version control (extra):     4.10.0

  Platform:
    Timestamp:                   2024-06-17T18:17:11Z
    Host:                        Darwin 20.2.0 arm64
    CMake:                       3.20.3
    CMake generator:             Unix Makefiles
    CMake build tool:            /usr/bin/make
    Configuration:               Release

  CPU/HW features:
    Baseline:                    NEON FP16
    Dispatched code generation:  NEON_DOTPROD NEON_FP16 NEON_BF16
      requested:                 NEON_FP16 NEON_BF16 NEON_DOTPROD
      NEON_DOTPROD (1 files):    + NEON_DOTPROD
      NEON_FP16 (2 files):       + NEON_FP16
      NEON_BF16 (0 files):       + NEON_BF16

  C/C++:
    Built as dynamic libs?:      NO
    C++ standard:     